### Carregando o modelo

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import json
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [11]:
from keras.models import load_model

In [12]:
def loadModel():
    name = input('escreva o nome do modelo a ser carregado:')
    savedModel = load_model(name+'.h5')
    with open('wordIndex.json') as f:
        dicto = json.load(f)
    savedModel.summary()
    return savedModel, dicto

In [13]:
tokenizer = Tokenizer()
model, tokenizer.word_index = loadModel()

escreva o nome do modelo a ser carregado:modelo96
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2000, 100)         8998800   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 9,116,177
Trainable params: 117,377
Non-trainable params: 8,998,800
_________________________________________________________________


### Notícias analisadas

In [58]:
sentence_t = '''O WhatsApp admitiu pela primeira vez que a eleição brasileira de 2018 teve uso de envios maciços de mensagens, com sistemas automatizados contratados de empresas.

“Na eleição brasileira do ano passado houve a atuação de empresas fornecedoras de envios maciços de mensagens, que violaram nossos termos de uso para atingir um grande número de pessoas”, afirmou Ben Supple, gerente de políticas públicas e eleições globais do WhatsApp, em palestra no Festival Gabo.

Em uma série de reportagens desde outubro do ano passado, a Folha revelou a contratação durante a campanha eleitoral de empresas de marketing que faziam envios maciços de mensagens políticas, usando de forma fraudulenta CPFs de idosos e até contratando agências estrangeiras.

Uma das reportagens noticiou que empresários apoiadores do então candidato Jair Bolsonaro (PSL) bancaram o disparo de mensagens em massa contra o petista Fernando Haddad, que foi derrotado e também acabou multado pelo TSE pelo impulsionamento irregular de conteúdo desfavorável ao seu então adversário. 

O TSE veda o uso de ferramentas de automatização, como os softwares de disparo em massa. Além disso, conforme mostrou a Folha, empresários contrataram disparos a favor e contra candidatos, sem declarar esses gastos à Justiça Eleitoral, o que configura o crime de caixa dois.

No mesmo evento, o executivo do WhatsApp condenou os grupos públicos da plataforma acessados por meio de links que distribuem conteúdo político, na maior parte das vezes relacionados ao governo Bolsonaro.

“Vemos esses grupos como tabloides sensacionalistas, onde as pessoas querem espalhar uma mensagem para uma plateia e normalmente divulgam conteúdo mais polêmico e problemático”, disse. “Nossa visão é: não entre nesses grupos grandes, com gente que você não conhece: saia desses grupos e os denuncie.”

O executivo do WhatsApp afirmou que a plataforma desencoraja “o uso dos grupos como listas de transmissão” de conteúdos, como ocorre com muitos grupos de apoiadores de políticos. “O WhatsApp foi criado para abrigar conversas orgânicas, entre famílias e amigos."

Indagado se o uso do WhatsApp por campanhas políticas violava as regras, ele afirmou: “Não viola desde que se respeitem todos os termos de uso [que vedam automação e envio massivo]. Todos estão sujeitos aos mesmos critérios, não importa se quem usa é um candidato à Presidência ou um camponês do interior da Índia.”

​Supple reconheceu a influência do aplicativo em processos eleitorais. ”Sabemos que eleições podem ser vencidas ou perdidas no WhatsApp”, disse. E afirmou que o WhatsApp despachou equipes para acompanhar as eleições de Índia, Indonésia e Parlamento Europeu no primeiro semestre.

Segundo o executivo, a plataforma já esperava que as eleições brasileiras de 2018 fossem palco de campanhas de desinformação. “Sempre soubemos que a eleição brasileira seria um desafio. Era uma eleição muito polarizada e as condições eram ideais para a disseminação de desinformação”, disse.

“No Brasil, muita gente usa o WhatsApp como fonte primária de informação e não tem meios para verificar a veracidade do conteúdo.” No entanto, apenas uma minoria comete irregularidades, afirma.

O executivo disse que a empresa vem adotando uma série de medidas para bloquear contas que violam as normas ao fazer envio automatizado ou maciço. E relatou que desde janeiro, quando o número de reencaminhamentos de uma mensagem foi limitado a cinco, o número total de reencaminhamentos caiu 25%. Supple afirmou que o WhatsApp tem banido 2 milhões de contas por mês.'''

In [59]:
sentence_f = "teste"

### Preprocessing

In [60]:
sentence_t = sentence_t.lower()

def remove_punct(raw_string):
    '''Eliminates punctuations and numbers from raw_string.'''
    translator = raw_string.maketrans({key:'' for key in string.punctuation.replace("-", '') + "“”‘’0123456789"})
    
    return raw_string.translate(translator)

sentence_t = remove_punct(sentence_t)

In [61]:
sentence_t

'o whatsapp admitiu pela primeira vez que a eleição brasileira de  teve uso de envios maciços de mensagens com sistemas automatizados contratados de empresas\n\nna eleição brasileira do ano passado houve a atuação de empresas fornecedoras de envios maciços de mensagens que violaram nossos termos de uso para atingir um grande número de pessoas afirmou ben supple gerente de políticas públicas e eleições globais do whatsapp em palestra no festival gabo\n\nem uma série de reportagens desde outubro do ano passado a folha revelou a contratação durante a campanha eleitoral de empresas de marketing que faziam envios maciços de mensagens políticas usando de forma fraudulenta cpfs de idosos e até contratando agências estrangeiras\n\numa das reportagens noticiou que empresários apoiadores do então candidato jair bolsonaro psl bancaram o disparo de mensagens em massa contra o petista fernando haddad que foi derrotado e também acabou multado pelo tse pelo impulsionamento irregular de conteúdo desfa

In [62]:
sentence_f = sentence_f.lower()

def remove_punct(raw_string):
    '''Eliminates punctuations and numbers from raw_string.'''
    translator = raw_string.maketrans({key:'' for key in string.punctuation.replace("-", '') + "“”‘’0123456789"})  

    return raw_string.translate(translator)

sentence_f = remove_punct(sentence_f)

### Classificação Trues

In [63]:
lista_t = []
lista_t.append(sentence_t)
lista_t

['o whatsapp admitiu pela primeira vez que a eleição brasileira de  teve uso de envios maciços de mensagens com sistemas automatizados contratados de empresas\n\nna eleição brasileira do ano passado houve a atuação de empresas fornecedoras de envios maciços de mensagens que violaram nossos termos de uso para atingir um grande número de pessoas afirmou ben supple gerente de políticas públicas e eleições globais do whatsapp em palestra no festival gabo\n\nem uma série de reportagens desde outubro do ano passado a folha revelou a contratação durante a campanha eleitoral de empresas de marketing que faziam envios maciços de mensagens políticas usando de forma fraudulenta cpfs de idosos e até contratando agências estrangeiras\n\numa das reportagens noticiou que empresários apoiadores do então candidato jair bolsonaro psl bancaram o disparo de mensagens em massa contra o petista fernando haddad que foi derrotado e também acabou multado pelo tse pelo impulsionamento irregular de conteúdo desf

In [64]:
maxlen = 2000
sequences_t = tokenizer.texts_to_sequences(lista_t)
padded_t = pad_sequences(sequences_t, maxlen=maxlen)

In [65]:
model.predict(padded_t)

array([[0.9538282]], dtype=float32)

In [66]:
(model.predict(padded_t) >=0.5).astype(int)

array([[1]])

### Classificação Fakes

In [22]:
lista_f = []
lista_f.append(sentence_f)
lista_f

['teste']

In [23]:
maxlen = 2000
sequences_f = tokenizer.texts_to_sequences(lista_f)
padded_f = pad_sequences(sequences_f, maxlen=maxlen)

In [24]:
model.predict(padded_f)

array([[0.00948888]], dtype=float32)

In [25]:
(model.predict(padded_f) >=0.5).astype(int)

array([[0]])